In [4]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

# Common imports
import numpy as np
import os

#importo libreria panda per leggere ed elaborare csv
import pandas as pd

# to make this notebook's output stable across runs
np.random.seed(42)


data=pd.read_csv('/Users/sgira/Downloads/ProgettoMLDM/ProgettoMLDM-main/Dataset/dataset_clean.csv')
train_data=pd.read_csv('/Users/sgira/Downloads/ProgettoMLDM/ProgettoMLDM-main/Dataset/train_data.csv')
test_data=pd.read_csv('/Users/sgira/Downloads/ProgettoMLDM/ProgettoMLDM-main/Dataset/test_data.csv')
y_train=pd.read_csv('/Users/sgira/Downloads/ProgettoMLDM/ProgettoMLDM-main/Dataset/train_y.csv')
y_test=pd.read_csv('/Users/sgira/Downloads/ProgettoMLDM/ProgettoMLDM-main/Dataset/test_y.csv')

In [5]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

#dataset=data.drop('Status',axis=1)
#y=data['Status']

#x_train, x_test, y_train, y_test = train_test_split(dataset, y, train_size=0.8)


In [43]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

#bagging con 1000 alberi di decisione
bag_clf = BaggingClassifier(
    estimator=DecisionTreeClassifier(ccp_alpha=0.0001, criterion='entropy', max_depth=20, random_state=10, min_samples_leaf=4, min_samples_split=6), n_estimators=1000,
    max_samples=30000, bootstrap=True, random_state=10, n_jobs=-1)
bag_clf.fit(train_data, np.ravel(y_train))
y_pred = bag_clf.predict(test_data)
print(accuracy_score(y_test, y_pred))


0.8749827085350671


In [35]:
tree_clf = DecisionTreeClassifier(ccp_alpha=0.0001, criterion='entropy', max_depth=20, random_state=10, min_samples_leaf=4, min_samples_split=6)
tree_clf.fit(train_data, np.ravel(y_train))

tree_clf.score(test_data, y_test)


0.8714206667588879

In [44]:
#bagging con out-of-bag evaluation
outbag_clf = BaggingClassifier(
    DecisionTreeClassifier(ccp_alpha=0.0001, criterion='entropy', max_depth=20, random_state=10, min_samples_leaf=4, min_samples_split=6), n_estimators=1000,
    max_samples=30000, bootstrap=True, oob_score=True, random_state=10, n_jobs=-1)
outbag_clf.fit(train_data, np.ravel(y_train))
outbag_clf.oob_score_

0.8709666263185198

In [31]:
#Adaboost con 200 alberi di decisione di un solo livello
from sklearn.ensemble import AdaBoostClassifier

ada_clf = AdaBoostClassifier(
    DecisionTreeClassifier(max_depth=1), n_estimators=200,
    algorithm="SAMME.R", learning_rate=0.5, random_state=42)
ada_clf.fit(train_data, np.ravel(y_train))
y_pred = ada_clf.predict(test_data)
print(accuracy_score(y_test, y_pred))

0.8662332272790151


In [12]:
from sklearn.preprocessing import StandardScaler,PolynomialFeatures,MinMaxScaler
from sklearn.model_selection import (
    GridSearchCV,
    StratifiedKFold,
    cross_val_score,
    train_test_split,
)
#adaboost con cross validation
scaled_dataset = StandardScaler().fit_transform(data)


abc = AdaBoostClassifier()

# n_estimators=150, random_state=10, algorithm='SAMME'
# Create the parameter grids
parameter_grid = {
     "n_estimators": [50, 100, 150],
     #"learning_rate": [.5, 1, 1.5],
     #"algorithm": ['SAMME', 'SAMME.R'],
    # 'random_state': [5, 10, 22, 50],
    'estimator': [#DecisionTreeClassifier(criterion='entropy', max_depth=2),
    #              DecisionTreeClassifier(criterion='entropy', max_depth=2),
    #              DecisionTreeClassifier(criterion='log_loss', max_depth=2),
    #              DecisionTreeClassifier(criterion='log_loss', max_depth=1),
                  # LinearSVC(dual=False, max_iter=200),
                  # LinearSVC(dual=False, max_iter=200, C = 10),
                  # LinearSVC(dual=False, max_iter=200, C = 50),
    #              SVC(kernel='rbf', max_iter=200),
    #              SVC(kernel='rbf', gamma='auto', max_iter=200),
    #              SVC(kernel='sigmoid', max_iter=200)
                  DecisionTreeClassifier(criterion='log_loss', max_depth=5,
                                         ccp_alpha=0.0001,),
                  DecisionTreeClassifier(criterion='entropy', max_depth=5,
                                         ccp_alpha=0.0001),
                  DecisionTreeClassifier(criterion='gini', max_depth=5,
                                         ccp_alpha=0.0001),
                  ]
}

# Create Stratified folds
# primi tentativi n_splits = 5 per limitare i tempi di esecuzione della gridSearch poi aumentato
cross_validation = StratifiedKFold(n_splits=5)
cross_validation.get_n_splits(data, y)

# Create the scoring dictionary
SCORING = {
    "accuracy": "accuracy",
    "balanced_accuracy": "balanced_accuracy",
    "f1": "f1_macro",
}

# Create and fit the GridSearchCV
grid_search = GridSearchCV(
    estimator=abc,
    param_grid=parameter_grid,
    # cv=cross_validation,
    verbose=3,
    scoring=SCORING,
    return_train_score=True,
    refit="balanced_accuracy",
    # error_score='raise'
)

grid_search.fit(scaled_dataset, y)

Fitting 5 folds for each of 9 candidates, totalling 45 fits
[CV 1/5] END estimator=DecisionTreeClassifier(ccp_alpha=0.0001, criterion='log_loss', max_depth=5), n_estimators=50; accuracy: (train=1.000, test=1.000) balanced_accuracy: (train=1.000, test=1.000) f1: (train=1.000, test=1.000) total time=   0.0s
[CV 2/5] END estimator=DecisionTreeClassifier(ccp_alpha=0.0001, criterion='log_loss', max_depth=5), n_estimators=50; accuracy: (train=1.000, test=1.000) balanced_accuracy: (train=1.000, test=1.000) f1: (train=1.000, test=1.000) total time=   0.0s
[CV 3/5] END estimator=DecisionTreeClassifier(ccp_alpha=0.0001, criterion='log_loss', max_depth=5), n_estimators=50; accuracy: (train=1.000, test=1.000) balanced_accuracy: (train=1.000, test=1.000) f1: (train=1.000, test=1.000) total time=   0.0s
[CV 4/5] END estimator=DecisionTreeClassifier(ccp_alpha=0.0001, criterion='log_loss', max_depth=5), n_estimators=50; accuracy: (train=1.000, test=1.000) balanced_accuracy: (train=1.000, test=1.000) f

[CV 5/5] END estimator=DecisionTreeClassifier(ccp_alpha=0.0001, max_depth=5), n_estimators=50; accuracy: (train=1.000, test=1.000) balanced_accuracy: (train=1.000, test=1.000) f1: (train=1.000, test=1.000) total time=   0.1s
[CV 1/5] END estimator=DecisionTreeClassifier(ccp_alpha=0.0001, max_depth=5), n_estimators=100; accuracy: (train=1.000, test=1.000) balanced_accuracy: (train=1.000, test=1.000) f1: (train=1.000, test=1.000) total time=   0.1s
[CV 2/5] END estimator=DecisionTreeClassifier(ccp_alpha=0.0001, max_depth=5), n_estimators=100; accuracy: (train=1.000, test=1.000) balanced_accuracy: (train=1.000, test=1.000) f1: (train=1.000, test=1.000) total time=   0.1s
[CV 3/5] END estimator=DecisionTreeClassifier(ccp_alpha=0.0001, max_depth=5), n_estimators=100; accuracy: (train=1.000, test=1.000) balanced_accuracy: (train=1.000, test=1.000) f1: (train=1.000, test=1.000) total time=   0.1s
[CV 4/5] END estimator=DecisionTreeClassifier(ccp_alpha=0.0001, max_depth=5), n_estimators=100; a

GridSearchCV(estimator=AdaBoostClassifier(),
             param_grid={'estimator': [DecisionTreeClassifier(ccp_alpha=0.0001,
                                                              criterion='log_loss',
                                                              max_depth=5),
                                       DecisionTreeClassifier(ccp_alpha=0.0001,
                                                              criterion='entropy',
                                                              max_depth=5),
                                       DecisionTreeClassifier(ccp_alpha=0.0001,
                                                              max_depth=5)],
                         'n_estimators': [50, 100, 150]},
             refit='balanced_accuracy', return_train_score=True,
             scoring={'accuracy': 'accuracy',
                      'balanced_accuracy': 'balanced_accuracy',
                      'f1': 'f1_macro'},
             verbose=3)

In [13]:
print("Best score: {}".format(grid_search.best_score_))
print("Best parameters: {}".format(grid_search.best_params_))

best_abc = grid_search.best_estimator_
best_abc

Best score: 1.0
Best parameters: {'estimator': DecisionTreeClassifier(ccp_alpha=0.0001, criterion='log_loss', max_depth=5), 'n_estimators': 50}


AdaBoostClassifier(estimator=DecisionTreeClassifier(ccp_alpha=0.0001,
                                                    criterion='log_loss',
                                                    max_depth=5))

In [14]:
my_model = best_abc
my_model.fit(scaled_dataset, y)
my_model.score(scaled_dataset, y)

1.0

In [2]:
#Xgboost
try:
    import xgboost
except ImportError as ex:
    print("Error: the xgboost library is not installed.")
    xgboost = None

In [8]:
from sklearn.metrics import mean_squared_error

if xgboost is not None:  # not shown in the book
    xgb_reg = xgboost.XGBClassifier(random_state=10, n_estimators=200,objective='binary:logistic')
    xgb_reg.fit(train_data, np.ravel(y_train))
    y_pred = xgb_reg.predict(test_data)
    val_error = mean_squared_error(y_test, y_pred) 
    print("Validation RMSE:", val_error)     
   

Validation RMSE: 0.12550145248305436


In [9]:
if xgboost is not None:  # not shown in the book
    xgb_reg.fit(train_data, np.ravel(y_train),
                eval_set=[(test_data, y_test)], early_stopping_rounds=2)
    y_pred = xgb_reg.predict(test_data)
    val_error = mean_squared_error(y_test, y_pred)  # Not shown
    print("Validation MSE:", val_error)            # Not shown

[0]	validation_0-logloss:0.45484
[1]	validation_0-logloss:0.41554
[2]	validation_0-logloss:0.39258
[3]	validation_0-logloss:0.37772
[4]	validation_0-logloss:0.36607
[5]	validation_0-logloss:0.35801


C:\Users\sgira\anaconda3\Lib\site-packages\xgboost\sklearn.py:885: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[6]	validation_0-logloss:0.35269
[7]	validation_0-logloss:0.34919
[8]	validation_0-logloss:0.34740
[9]	validation_0-logloss:0.34492
[10]	validation_0-logloss:0.34329
[11]	validation_0-logloss:0.34216
[12]	validation_0-logloss:0.34049
[13]	validation_0-logloss:0.34014
[14]	validation_0-logloss:0.33915
[15]	validation_0-logloss:0.33754
[16]	validation_0-logloss:0.33725
[17]	validation_0-logloss:0.33602
[18]	validation_0-logloss:0.33537
[19]	validation_0-logloss:0.33396
[20]	validation_0-logloss:0.33315
[21]	validation_0-logloss:0.33253
[22]	validation_0-logloss:0.33261
[23]	validation_0-logloss:0.33187
[24]	validation_0-logloss:0.33171
[25]	validation_0-logloss:0.33158
[26]	validation_0-logloss:0.33090
[27]	validation_0-logloss:0.33061
[28]	validation_0-logloss:0.33064
[29]	validation_0-logloss:0.33039
[30]	validation_0-logloss:0.33046
[31]	validation_0-logloss:0.33034
[32]	validation_0-logloss:0.33036
[33]	validation_0-logloss:0.32985
[34]	validation_0-logloss:0.32966
[35]	validation_0-